# Regularized Classification on Titanic Dataset

We are going to use a dataset from a Kaggle competition (https://www.kaggle.com/c/titanic/data)
 
### Dataset description

>The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew.  This sensational tragedy shocked the international community and led to better safety regulations for ships.

>One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew.  Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

>In this contest, we ask you to complete the analysis of what sorts of people were likely to survive. 

From the competition [homepage](http://www.kaggle.com/c/titanic-gettingStarted).



# TO DO: put your ID number ("numero di matricola") to be used as seed for splitting the data into training and test

In [ ]:
import numpy as np

#put here your ``numero di matricola''
IDnumber = 1110975
np.random.seed(IDnumber)

In [ ]:
# let's load library for plotting
%matplotlib inline  
import matplotlib.pyplot as plt

## Data Preprocessing

Load the data from a .csv file. In this notebook we use the pandas packages, since it provides useful functions to clean the data. In particular, it allows us to remove samples with missing data, that we do below. We also plot some descriptions of columns, check the pandas documentation for 'describe()' if you want to know more.

In [ ]:
# let's load pands and numpy
import pandas as pd

# this time we use pandas to load and clean the dataset

# read the data from the cvs file
df = pd.read_csv("titanicData.csv")

# remove columns 'Ticket', 'Cabin', and 'Name' from the data since they are not relevant
df = df.drop(['Ticket','Cabin','Name'], axis=1)
# remove samples with missing values
df = df.dropna() 
# let's see some statistics about the data 
df.describe()

Now we create data matrices: many of the features (columns of indices 0,1,3,4,6 in Xcat below) are categorical, so we need to encode them with ***indicator matrices***. That is, if a feature can take $\ell$ different values $v_1,\dots,v_{\ell}$, we create $\ell$ indicator (0-1) features $I_1,\dots,I_{\ell}$, such that $I_{j} = 1$ if and only if the value of the feature is $v_j$. This can be done in Python by first encode a feature with integers with LabelEncoder() and then obtain the indicator variables with OneHotEncoder().

In [ ]:
#df.values contains the data, both the values of instances and the value of the label
Data = df.values
# the matrix including the categorical data is given by columns from the second one 
X_categorical = Data[:,2:]
# the target value (class) is in the first column
Y = Data[:,1]

print list(df)

# get the number d of features of each sample
d = X_categorical.shape[1]

# get the number m of samples
m = X_categorical.shape[0]

#let's see what the number of samples is
print "Number of samples: {}".format(m)

#now encode categorical variables using integers and one-hot-encoder

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder()

# encode the first column of the data matrix into indicator variables

X_tmp = label_encoder.fit_transform(X_categorical[:,0])
X_tmp = X_tmp.reshape(X_tmp.shape[0],1)
X = onehot_encoder.fit_transform(X_tmp[:,0].reshape(-1,1)).toarray()

# repeat for the other categorical input variables, which have indices 1, 3, 4, and 6 in the X_categorical

index_categorical = [1,3,4,6]

for i in range(1,7):
    if i in index_categorical:
        X_tmp = label_encoder.fit_transform(X_categorical[:,i])
        X_tmp = X_tmp.reshape(X_tmp.shape[0],1)
        X_tmp = onehot_encoder.fit_transform(X_tmp[:,0].reshape(-1,1)).toarray()
        X = np.hstack((X,X_tmp))
    else:
        X_tmp = X_categorical[:,i]
        X_tmp = X_tmp.reshape(X_tmp.shape[0],1)
        X = np.hstack((X,X_tmp))

The class labels are already 0-1, so we can use them directly.

In [ ]:
# properly encode the target labels
Y = label_encoder.fit_transform(Y)
K = max(Y) + 1 # number of classes

print "Number of classes: "+str(K)

Given $m$ total data points, keep $m\_training = 50$ data points as data for ***training and validation*** and $m\_test = m - m\_training$ as test data. Splitting is random, using as seed your ID number. Make sure that the training set contains at least 10 instances from each class.If it does not, modify the code so to apply a random
permutation (or the same permutation multiple times) to the samples until this happens.

In [ ]:
# Split data into training and validation data

# load a package which is useful for the training-test splitting
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

# number of samples
m = np.shape(X)[0]

#Divide in training and test: make sure that your training set
#contains at least 10 elements from class 1 and at least 10 elements
#from class -1! If it does not, modify the code so to apply more random
#permutations (or the same permutation multiple times) until this happens.

permutation = np.random.permutation(m)
X = X[permutation]
Y = Y[permutation]

m_training = 50  # use 50 samples for training + validation...
m_test = m-m_training # and the rest for testing

# test_size is the proportion of samples in the test set
X_training, X_test, Y_training, Y_test = train_test_split(X, Y, test_size =float(m_test)/float(m), random_state = IDnumber)

print Y_training

m_training = X_training.shape[0]
m_test = X_test.shape[0]

#let's see what the fraction of ones in the entire dataset is
print float(sum(Y_training)+sum(Y_test))/float(m_training+m_test)

Standardize the data to have zero-mean and unit variance (columnwise):

In [ ]:
# Standardize the Features Matrix
from sklearn import preprocessing

#let's use the standard scaling; we degine the scaling for the entire dataset
scaler = preprocessing.StandardScaler().fit(X)

#let's apply the scaling to the training set
X_training = scaler.transform(X_training)
#let's apply the scaling to the test set
X_test = scaler.transform(X_test)

### Perform Logistic Regression

We now perform logistic regression using the function provided by Scikit-learn.

Note: as provided by Scikit-learn, logistic regression is always implemented using regularization. However, the impact of regularization can be dampened to have almost no regularization by changing the parameter $C$, which is the inverse of $\lambda$. Therefore to have no regularization, which is $\lambda = 0$ for the model seen in class, we need $C$ to have a large value. Here we fix $C = 100000000$.

[Note that the intercept is estimated in the model.]

For all our models we are going to use 10-fold cross validation to estimate the parameters (when needed) and/or estimate the validation error.

In [ ]:
from sklearn import linear_model

# define a logistic regression model with very high C parameter -> low impact from regularization;
# there are many solvers available to obtain the solution to the logistic regression problem, we just pick
# one of them; 'cv' is the number of folds in cross-validation; we also specify l2 as regularization penalty,
# just to pick one; Cs contains the values of C to be tested and to pick from with validation. Here we
# are interested in only 1 value of C, and use cross-validation just to estimate the validation error
# in a same way as other models

reg = linear_model.LogisticRegressionCV(Cs=[100000000], solver='newton-cg',cv=10, penalty='l2')

#fit the model on training data
reg.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( reg.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds

CV_accuracies = np.divide(np.sum(reg.scores_[1],axis=0),10)

# let's print the average accuracies obtained for the various values of C

print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it

print "Best value of parameter C according to 10-fold Cross-Validation: {}".format( reg.C_[0] )

# let's store the best CV accuracy, and then print it
reg_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( reg_best_CV_accuracy )

Note that the logistic regression function in Scikit-learn has many optional parameters. Read the documentation if you want to understand what they do!

## TODO 1
### Learn the best model from Logistic Regression on the entire training set and examine coefficients (by printing and plotting them)

Note that you can use simply $linear\_model.LogisticRegression()$ (with the same parameters used in cross validation, of course not the ones that are meaningful only for cross-validation!), that does not use cross-validation, without passing the best value of $C$ (and then fit()).

In [ ]:
# let's define the Logistic Regression model
reg_full = linear_model.LogisticRegression(solver='newton-cg', penalty='l2')

# get the best model using the entire training dataset
reg_full.fit(X_training, Y_training)

# print the coefficients from the logistic regression model.
print "Coefficients obtained using the entire training set: {}".format( reg_full.coef_ )

# note that the intercept is not in coef_, it is in intercept_
print "Intercept from logistic regression: {}".format(reg_full.intercept_)

# Plot the coefficients
reg_coef = reg_full.coef_.reshape(reg_full.coef_.shape[1],)
plt.figure()
ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
width = 0.45       # the width of the bars
plt.bar(ind, reg_coef, width, color='r')
plt.xlabel('Coefficient Index')
plt.ylabel('Coefficient Value')
plt.title('Logistic Regression Coefficients')
plt.show()

## TODO 2

### Questions: How many coefficients do you get? Why? How many of them are equal to 0? (max 5 lines)

### ANSWER
I get 23 coefficients: since four features are categorical, we had to create new features, one for each value that such feature can assume. None of the coefficients are equal to zero but some of them (7 actually) have values very close to 0, in the order of $10^{-6}$ or $10^{-7}$, that is, all the features that do not appear in the figure above.

## TO DO 3
### Predict labels on training and test

- Compute the predicted labels on training and test data using reg.predict
 - Evaluate the accuracy using metrics.accuracy_score from scikit-learn (it returns the percentage of data correctly classified).
 - Evaluate the score used by logistic regression on training and test data using metrics.accuracy_score(). (Read the documentation!)

In [ ]:
from sklearn import metrics

# prediction on training data
Y_training_prediction_LR = reg.predict(X_training)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn for training dataset and print it
accuracy_training = metrics.accuracy_score(Y_training, Y_training_prediction_LR)

# prediction on test data
Y_test_prediction_LR = reg.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn for test dataset
accuracy_test = metrics.accuracy_score(Y_test, Y_test_prediction_LR)

print "Accuracy on training set: ", accuracy_training
print "Accuracy on test set: ", accuracy_test

## TODO 4
### Use L2 regularized logistic regression with cross-validation

We perform the L2 regularization for different values of the regularization parameter $C$, and use the Scikit-learn function to perform cross-validation (CV).

In L2 regularized logistic regression, the following L2 regularization term is added to the loss:

$$
    \lambda \sum_{i=1}^d w_i^2
$$

The parameter $C$ used by Scikit learn corresponds to the inverse of $\lambda$, that is $C = \frac{1}{\lambda}$.

Note: the CV in Scikit-learn is by default a *stratified* CV, that means that data is split into train-validation while maintaining the proportion of different classes in each fold.

In the code below:
- use LogisticRegressionCV() to select the best value of C with a 10-fold CV with L2 penalty;
- use LogisticRegression() to learn the best model for the best C with L2 penalty on the entire training set

Note that LogisticRegressionCV() picks some default values of C to try, but you may need to pass some other values in case for your dataset you need to explore a different interval of values. This applies every time that you use LogisticRegressionCV().

In [ ]:
#define the model using LogisticRegressionCV passing an appropriate solver, cv value, and choice of penalty
folds=10
regL2 = linear_model.LogisticRegressionCV(cv=folds, penalty='l2', solver='newton-cg')

#fit the model on training data
regL2.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( regL2.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds
CV_accuracies = [sum(regL2.scores_[1][i])/folds for i in range(folds)]

# let's print the average accuracies obtained for the various values of C
print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it
regL2_best_C = regL2.C_[0]
print "Best value of parameters C as identified by cross-validation: {}".format(regL2_best_C)

# let's store the best CV accuracy, and then print it
regL2_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( regL2_best_CV_accuracy )

#define the model using the best C and an appropriate solver
regL2_full = linear_model.LogisticRegression(C=regL2_best_C, penalty='l2', solver='newton-cg')

#fit the model using the best C on the entire training set
regL2_full.fit(X_training, Y_training)

### TODO 5: print and plot the coefficients from logistic regression and the regularized version.

In [ ]:
#print the coefficients from logistic regression
print "Coefficients from logistic regression: {}".format(reg_coef)

#print the coefficients from L2 regularized logistic regression
print "Coefficients from L2 regularized logistic regression: {}".format(regL2_full.coef_)

# note that the intercept is not in coef_, it is in intercept_
print "Intercept from logistic regression: {}".format(reg_full.intercept_)
print "Intercept from L2 regularized logistic regression: {}".format( regL2_full.intercept_ )

# Plot the coefficients
regL2_full_coef = regL2_full.coef_.reshape(regL2_full.coef_.shape[1],)
ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
width = 0.35       # the width of the bars
fig, ax = plt.subplots()

rects1 = ax.bar(ind, reg_coef, width, color='r')
rects2 = ax.bar(ind + width, regL2_full_coef, width, color='y')
ax.legend((rects1[0], rects2[0]), ('Log Regr', 'Log Regr + L2 Regul'))
plt.xlabel('Coefficient Idx')
plt.ylabel('Coefficient Value')
plt.title('Logistic Regression Coefficients: Standard and Regularized Version')
plt.show()

### TODO 6: how do the coefficients from L2 regularization compare to the ones from logistic regression? (max 5 lines)

### ANSWER
Most of the coefficients from L2 regularization have (absolute) values greater than the ones from logistic regression. Even the coefficients that was very close to 0 are now greater of two order.

### TODO 7: obtain classification accuracy on training and test data for the L2 regularized model

In [ ]:
#now get training and test error and print training and test accuracy

# predictions on training data 
Y_training_prediction_LR_L2 = regL2_full.predict(X_training)

# predictions on test data 
Y_test_prediction_LR_L2 = regL2_full.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on training data
print "Training Accuracy:", metrics.accuracy_score(Y_training, Y_training_prediction_LR_L2)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on test data
print "Test Accuracy:", metrics.accuracy_score(Y_test, Y_test_prediction_LR_L2)

### TODO 8: how does accuracy compare to logistic regression? Comment (max 5 lines)

### ANSWER
The accuracies both in training and test data sre basically the same, altough we can register a little increase in test accuracy.

### TODO 9: Use L1 regularized logistic regression with cross-validation

We perform the L1 regularization for different values of the regularization parameter $C$, and use the Scikit-learn function to perform cross-validation (CV).

In L1 regularized logistic regression, the following L1 regularization term is added to the loss:

$$
    \lambda \sum_{i=1}^d |w_i|
$$

The parameter $C$ used by Scikit learn corresponds to the inverse of $\lambda$, that is $C = \frac{1}{\lambda}$.

Note: the CV in Scikit-learn is by default a *stratified* CV, that means that data is split into train-validation while maintaining the proportion of different classes in each fold.

In the code below:
- use LogisticRegressionCV() to select the best value of C with a 10-fold CV with L1 penalty;
- use LogisticRegression() to learn the best model for the best C with L1 penalty on the entire training set

Note: not all the solvers in LogisticRegressionCV() and LogisticRegression() can be used for L1 regularization! See the documentation and choose an appropriate solver.

In [ ]:
#define the model using LogisticRegressionCV passing an appropriate solver, cv value, and choice of penalty
regL1 = linear_model.LogisticRegressionCV(cv=folds, solver='liblinear', penalty='l1')

#fit the model on training data
regL1.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( regL1.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds

CV_accuracies = [sum(regL1.scores_[1][i])/folds for i in range(folds)]

# let's print the average accuracies obtained for the various values of C

print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it
regL1_best_C = regL1.C_[0]
print "The best value of parameter C as identified by cross-validation: {}".format(regL1.C_)

# let's store the best CV accuracy, and then print it
regL1_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( regL1_best_CV_accuracy )

#define the model using the best C and an appropriate solver
regL1_full = linear_model.LogisticRegression(C=regL1_best_C, solver='liblinear', penalty='l1')

#fit the model using the best C on the entire training set
regL1_full.fit(X_training, Y_training)

### TODO 10: plot the coefficients from logistic regression and the regularized version.

In [ ]:
#print the coefficients from logistic regression
print "Coefficients from logistic regression: {}".format(reg_coef)
print "Intercept from logistic regression: {}".format(reg_full.intercept_)

#print the coefficients from L2 regularized logistic regression
print "Coefficients from L2 regularized logistic regression: {}".format(regL2_full.coef_)
print "Intercept from L2 regularized logistic regression: {}".format(regL2_full.intercept_)

#print the coefficients from L1 regularized logistic regression
print "Coefficients from L1 regularized logistic regression: {}".format(regL1_full.coef_)
print "Intercept from L1 regularized logistic regression: {}".format(regL1_full.intercept_)

# Plot the coefficients
regL1_full_coef = regL1_full.coef_.reshape(regL1_full.coef_.shape[1],)

ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
width = 0.25       # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, reg_coef, width, color='r')
rects2 = ax.bar(ind + width, regL2_full_coef, width, color='y')
rects3 = ax.bar(ind + 2*width, regL1_full_coef, width, color='g')
ax.legend((rects1[0], rects2[0], rects3[0]), ('Log Regr', 'Log Regr + L2 Regul', 'Log Regr + L1 Regul'))
plt.xlabel('Coefficient Idx')
plt.ylabel('Coefficient Value')
plt.title('Logistic Regression Coefficients: Standard, Regularized L2, and Regularized L1 Version')
plt.show()

### TODO 11: how do the coefficients from L1 regularization compare to the ones from logistic regression and to the ones from L2 regularization? (max 5 lines)

### ANSWER
Now, with L1 regularization, most of the coefficients are set to zero. This is expected from the L1 penalty: its job compares to features selection setting unuseful predictors to zero. The shrinkage propery of L1 applies to all coefficients, so also the non-zero coefficients are affected by it and it results that those coefficients have values less than the values of the same coefficients obtained with the previous two method.

### TODO 12: obtain classification accuracy on training and test data for the best L1 regularized model

In [ ]:
#now get training and test error and print training and test accuracy

# predictions on training data 
Y_training_prediction_LR_L1 = regL1_full.predict(X_training)

# predictions on test data 
Y_test_prediction_LR_L1 = regL1_full.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on training data
print "Training Accuracy:", metrics.accuracy_score(Y_training, Y_training_prediction_LR_L1)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on test data
print "Test Accuracy:", metrics.accuracy_score(Y_test, Y_test_prediction_LR_L1)

### TODO 13: Which model would have picked using cross-validation? Why? Which model gives the best (estimated) generalization error? (max 5 lines)

### ANSWER
All models have basically the same generalization error on test data. That could mean that the first model is accurate enough with the data we have and we do not need to tune it with L2 or L1 penalties. Given the results the models seem equivalent, but I would choose the model with L1 penalty since it is (in general) faster and prevents overfitting (less predictors).

### TODO 14: use larger datasets for training set

Perform the same estimation procedures using different more points on the training data, that is fix $m\_training = 400$. You can simply copy and paste all the code above into the cell below.

In [ ]:
print "REDO EVERYTHING WITH 400 SAMPLES IN THE TRAINING SET\n"
print "DATA SPLITTING AND STANDARDIZATION...\n"
m_training = 400  # use 400 samples for training + validation...
m_test = m-m_training # and the rest for testing

# test_size is the proportion of samples in the test set
X_training, X_test, Y_training, Y_test = train_test_split(X, Y, test_size =float(m_test)/float(m), random_state = IDnumber)

#print Y_training

m_training = X_training.shape[0]
m_test = X_test.shape[0]

#let's see what the fraction of ones in the entire dataset is
#print float(sum(Y_training)+sum(Y_test))/float(m_training+m_test)

#let's use the standard scaling; we degine the scaling for the entire dataset
scaler = preprocessing.StandardScaler().fit(X)

#let's apply the scaling to the training set
X_training = scaler.transform(X_training)
#let's apply the scaling to the test set
X_test = scaler.transform(X_test)

print "LOGISTIC REGRESSION..."
reg = linear_model.LogisticRegressionCV(Cs=[100000000], solver='newton-cg',cv=10, penalty='l2')

#fit the model on training data
reg.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( reg.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds

CV_accuracies = np.divide(np.sum(reg.scores_[1],axis=0),10)

# let's print the average accuracies obtained for the various values of C

print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it

print "Best value of parameter C according to 10-fold Cross-Validation: {}".format( reg.C_[0] )

# let's store the best CV accuracy, and then print it
reg_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( reg_best_CV_accuracy )

# let's define the Logistic Regression model
reg_full = linear_model.LogisticRegression(solver='newton-cg', penalty='l2')

# get the best model using the entire training dataset
reg_full.fit(X_training, Y_training)

# print the coefficients from the logistic regression model.
#print "Coefficients obtained using the entire training set: {}".format( reg_full.coef_ )

# note that the intercept is not in coef_, it is in intercept_

#print "Intercept: {}".format( reg_full.intercept_ )

# Plot the coefficients
reg_coef = reg_full.coef_.reshape(reg_full.coef_.shape[1],)
#plt.figure()
#ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
#width = 0.45       # the width of the bars
#plt.bar(ind, reg_coef, width, color='r')
#plt.xlabel('Coefficient Index')
#plt.ylabel('Coefficient Value')
#plt.title('Logistic Regression Coefficients')
#plt.show()

# prediction on training data
Y_training_prediction_LR = reg.predict(X_training)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn for training dataset and print it
accuracy_training = metrics.accuracy_score(Y_training, Y_training_prediction_LR)

# prediction on test data
Y_test_prediction_LR = reg.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn for test dataset
accuracy_test = metrics.accuracy_score(Y_test, Y_test_prediction_LR)

print "Accuracy on training set: ", accuracy_training
print "Accuracy on test set: ", accuracy_test

print "\nL2 REGULARIZED LOGISTIC REGRESSION..."
#define the model using LogisticRegressionCV passing an appropriate solver, cv value, and choice of penalty
folds=10
regL2 = linear_model.LogisticRegressionCV(cv=folds, penalty='l2', solver='newton-cg')

#fit the model on training data
regL2.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( regL2.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds
CV_accuracies = [sum(regL2.scores_[1][i])/folds for i in range(folds)]

# let's print the average accuracies obtained for the various values of C
print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it
regL2_best_C = regL2.C_[0]
print "Best value of parameters C as identified by cross-validation: {}".format(regL2_best_C)

# let's store the best CV accuracy, and then print it
regL2_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( regL2_best_CV_accuracy )

#define the model using the best C and an appropriate solver
regL2_full = linear_model.LogisticRegression(C=regL2_best_C, penalty='l2', solver='newton-cg')

#fit the model using the best C on the entire training set
regL2_full.fit(X_training, Y_training)

#print the coefficients from logistic regression
#print "Coefficients from logistic regression: {}".format(reg_coef)

#print the coefficients from L2 regularized logistic regression
#print "Coefficients from L2 regularized logistic regression: {}".format(regL2_full.coef_)

# note that the intercept is not in coef_, it is in intercept_
#print "Intercept from logistic regression: {}".format(reg_full.intercept_)
#print "Intercept from L2 regularized logistic regression: {}".format( regL2_full.intercept_ )

# Plot the coefficients
regL2_full_coef = regL2_full.coef_.reshape(regL2_full.coef_.shape[1],)
#ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
#width = 0.35       # the width of the bars
#fig, ax = plt.subplots()
#
#rects1 = ax.bar(ind, reg_coef, width, color='r')
#rects2 = ax.bar(ind + width, regL2_full_coef, width, color='y')
#ax.legend((rects1[0], rects2[0]), ('Log Regr', 'Log Regr + L2 Regul'))
#plt.xlabel('Coefficient Idx')
#plt.ylabel('Coefficient Value')
#plt.title('Logistic Regression Coefficients: Standard and Regularized Version')
#plt.show()

#now get training and test error and print training and test accuracy

# predictions on training data 
Y_training_prediction_LR_L2 = regL2_full.predict(X_training)

# predictions on test data 
Y_test_prediction_LR_L2 = regL2_full.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on training data
print "Training Accuracy:", metrics.accuracy_score(Y_training, Y_training_prediction_LR_L2)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on test data
print "Test Accuracy:", metrics.accuracy_score(Y_test, Y_test_prediction_LR_L2)

print "\nL1 REGULARIZED LOGISTIC REGRESSION..."

#define the model using LogisticRegressionCV passing an appropriate solver, cv value, and choice of penalty
regL1 = linear_model.LogisticRegressionCV(cv=folds, solver='liblinear', penalty='l1')

#fit the model on training data
regL1.fit(X_training, Y_training)

# the attribute 'Cs_' contains ALL the values of C evaluated in cross-validation;
# let's print them
print "Values of parameter C tried in 10-fold Cross-Validation: {}".format( regL1.Cs_ )

# the attribute 'scores_' contains the accuracy obtained in each fold, for each value 
# of C tried; we now compute the average accuracy across the 10 folds

CV_accuracies = [sum(regL1.scores_[1][i])/folds for i in range(folds)]

# let's print the average accuracies obtained for the various values of C

print "Accuracies obtained for the different values of C with 10-fold Cross-Validation: {}".format( CV_accuracies )

# the attribute 'C_' contains the best value of C as identified by cross-validation;
# let's print it
regL1_best_C = regL1.C_[0]
print "The best value of parameter C as identified by cross-validation: {}".format(regL1.C_)

# let's store the best CV accuracy, and then print it
regL1_best_CV_accuracy = max(CV_accuracies)
print "10-fold Cross-Validation accuracies obtained with the best value of parameter C: {}".format( regL1_best_CV_accuracy )

#define the model using the best C and an appropriate solver
regL1_full = linear_model.LogisticRegression(C=regL1_best_C, solver='liblinear', penalty='l1')

#fit the model using the best C on the entire training set
regL1_full.fit(X_training, Y_training)

#now get training and test error and print training and test accuracy

# predictions on training data 
Y_training_prediction_LR_L1 = regL1_full.predict(X_training)

# predictions on test data 
Y_test_prediction_LR_L1 = regL1_full.predict(X_test)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on training data
print "Training Accuracy:", metrics.accuracy_score(Y_training, Y_training_prediction_LR_L1)

# compute accuracy as suggested above using metrics.accuracy_score from scikit-learn on test data
print "Test Accuracy:", metrics.accuracy_score(Y_test, Y_test_prediction_LR_L1)

print "\n"

#print the coefficients from logistic regression
print "Coefficients from logistic regression: {}".format(reg_coef)
print "Intercepts from logistic regression: {}\n".format(reg_full.intercept_)

#print the coefficients from L2 regularized logistic regression
print "Coefficients from L2 regularized logistic regression: {}".format(regL2_full.coef_)
print "Intercepts from L2 regularized logistic regression: {}\n".format(regL2_full.intercept_)

#print the coefficients from L1 regularized logistic regression
print "Coefficients from L1 regularized logistic regression: {}".format(regL1_full.coef_)
print "Intercepts from L1 regularized logistic regression: {}\n".format(regL1_full.intercept_)

# Plot the coefficients
regL1_full_coef = regL1_full.coef_.reshape(regL1_full.coef_.shape[1],)

ind = np.arange(1,len(reg_coef)+1)  # the x locations for the groups
width = 0.25       # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, reg_coef, width, color='r')
rects2 = ax.bar(ind + width, regL2_full_coef, width, color='y')
rects3 = ax.bar(ind + 2*width, regL1_full_coef, width, color='g')
ax.legend((rects1[0], rects2[0], rects3[0]), ('Log Regr', 'Log Regr + L2 Regul', 'Log Regr + L1 Regul'))
plt.xlabel('Coefficient Idx')
plt.ylabel('Coefficient Value')
plt.title('Logistic Regression Coefficients: Standard, Regularized L2, and Regularized L1 Version')
plt.show()